In [1]:
from utils import Neo4jConnection
import pandas as pd

In [4]:
statements = pd.read_csv("../neo4j_db/import/statements.csv")
property = pd.read_csv("../neo4j_db/import/property.csv")

In [5]:
len(statements)

141206853

In [8]:
property[property["property_id"]==793]

,property_id,en_label,en_description
569,793,significant event,significant or notable events associated with ...


In [6]:
merged_df = statements.merge(property,how="left",left_on="edge_property_id",right_on="property_id")

In [7]:
merged_df.en_label = merged_df.en_label.str.replace(" ","_")

In [12]:
len(property.en_label.unique())

6985

In [20]:
property["en_label_text_clean"] = property.en_label.str.replace(" ","_").str.upper().str.replace("'","").str.replace('"','')

In [21]:
property.head()

,property_id,en_label,en_description,en_label_text_clean
0,6,head of government,"head of the executive power of this town, city...",HEAD_OF_GOVERNMENT
1,10,video,"relevant video. For images, use the property P...",VIDEO
2,14,traffic sign,"graphic symbol describing the item, used at th...",TRAFFIC_SIGN
3,15,route map,image of route map at Wikimedia Commons,ROUTE_MAP
4,16,highway system,system (or specific country specific road type...,HIGHWAY_SYSTEM


In [24]:
for index, row in property.iterrows():
    if index >5:
        break
    statements[statements["edge_property_id"]==row["property_id"]].to_csv("./edge_files/"+row["en_label_text_clean"]+".csv", index=False)
    

In [6]:
merged_df.to_csv("wikimedia_relations.csv", index = False)

In [4]:
merged_df.en_label = merged_df.en_label.str.replace(" ","_")

In [5]:
conn = Neo4jConnection("bolt://localhost:7687","neo4j","academic")

Verbindung zur Neo4j-Datenbank erfolgreich hergestellt.


In [6]:
edges = merged_df.iloc[:2].to_dict('records')

#conn.create_edges(edges)

In [13]:
query = """
            MATCH (source:Page {item_id: $source_item_id})
            MATCH (target:Page {item_id: $target_item_id})
            MERGE (source)-[:WIKIDATA_RELATION {edge_label: $edge_label}]->(target)
        """
with conn.driver.session() as session:
    session.run(query, source_item_id=20, target_item_id=46, edge_label = "test_2")

In [16]:
# WORKS!!

query = """
            UNWIND $edges AS edge
            MATCH (source:Page {item_id: edge.source_item_id})
            MATCH (target:Page {item_id: edge.target_item_id})
            MERGE (source)-[:WIKIDATA_RELATION {en_label: edge.en_label, en_description: edge.en_description, property_id: edge.property_id}]->(target)
        """

edges = [{"source_item_id":20, 
                                "target_item_id":46, 
                                "en_label" : "test_2", 
                                "en_description": "testi", 
                                "property_id":"test4"},
                                {"source_item_id":20, 
                                "target_item_id":46, 
                                "en_label" : "test_6", 
                                "en_description": "test3", 
                                "property_id":"test29"},
                                ]

with conn.driver.session() as session:
    session.run(query, edges = edges)

In [ ]:
conn = Neo4jConnection("bolt://localhost:7687","neo4j","academic")


query = """
            UNWIND ['HEAD_OF_GOVERNMENT','HIGHWAY_SYSTEM'] as outer
            LOAD CSV WITH HEADERS FROM 'file:///'+ outer + '.csv' AS row
            MATCH (p1:Page {item_id: toInteger(row.source_item_id)}), (p2:Page {item_id: toInteger(row.target_item_id)})
            CREATE (p1)-[:outer {property_id: toInteger(row.edge_property_id)}]->(p2)
        """
with conn.driver.session() as session:
    session.run(query, source_item_id=20, target_item_id=46, edge_label = "test_2")

In [39]:
import os

In [138]:
property["en_label_text_clean"] = property.en_label.str.strip().str.upper().str.replace("'","").str.replace('"','').str.replace(" ","_").str.replace("/","_").str.replace("-","_").str.replace("(","_").str.replace(")","_").str.replace(".","_").str.replace("–","_").str.replace(",","_").str.replace("&","_")

In [47]:
# Creates separate files for each edge type
for index, row in property.iterrows():
    filepath = "../neo4j_db/import/edges/" + row["en_label_text_clean"]+".csv"
    if not os.path.isfile(filepath):
        df = statements[statements["edge_property_id"]==row["property_id"]]
        if len(df) > 0:
            print(filepath)
            df.to_csv(filepath, index=False)
    

../neo4j_db/import/edges/POSITION_PLAYED_ON_TEAM_-_SPECIALITY.csv
../neo4j_db/import/edges/STOCK_EXCHANGE.csv
../neo4j_db/import/edges/RADIO_FORMAT.csv
../neo4j_db/import/edges/PATRON_SAINT.csv
../neo4j_db/import/edges/SEAL_DESCRIPTION.csv
../neo4j_db/import/edges/LOCATED_IN_TIME_ZONE.csv
../neo4j_db/import/edges/SHOOTING_HANDEDNESS.csv
../neo4j_db/import/edges/FIELD_OF_THIS_OCCUPATION.csv
../neo4j_db/import/edges/TAXONOMIC_TYPE.csv
../neo4j_db/import/edges/DISTRIBUTION_FORMAT.csv
../neo4j_db/import/edges/REVIEW_SCORE_BY.csv
../neo4j_db/import/edges/ORIGINAL_NETWORK.csv
../neo4j_db/import/edges/ASTRONAUT_MISSION.csv
../neo4j_db/import/edges/PARTNER.csv
../neo4j_db/import/edges/INDUSTRY.csv
../neo4j_db/import/edges/CHARACTER_ROLE.csv
../neo4j_db/import/edges/FOUNDATIONAL_TEXT.csv
../neo4j_db/import/edges/DETERMINATION_METHOD.csv
../neo4j_db/import/edges/SAID_TO_BE_THE_SAME_AS.csv
../neo4j_db/import/edges/OPPOSITE_OF.csv
../neo4j_db/import/edges/COLOR.csv
../neo4j_db/import/edges/MEMBER_

In [52]:
# Erzeugt alle Kanten

conn = Neo4jConnection("bolt://localhost:7687","neo4j","academic")

for i, item in enumerate(os.listdir("../neo4j_db/import/edges")):
    if i>3:
        break
    query_0 = f"""
            LOAD CSV WITH HEADERS FROM 'file:///edges/'+ '{item}' AS row"""
    query_1 = """
            MATCH (p1:Page {item_id: toInteger(row.source_item_id)}), (p2:Page {item_id: toInteger(row.target_item_id)})
            CREATE (p1)-"""
    query_2 = f"""[:{item.replace(".csv","")} """ 
    query_3 = """{property_id: toInteger(row.edge_property_id)}]->(p2)"""
    query= query_0 + query_1 + query_2 + query_3
    with conn.driver.session() as session:
        session.run(query)

Verbindung zur Neo4j-Datenbank erfolgreich hergestellt.


In [148]:
# Helperfunctions

def create_composite_index_query_on_id(relation, id):
    query = f"""CREATE INDEX composite_range_rel_index_{id} IF NOT EXISTS FOR ()-[r:{relation}]-() ON (r.{id})"""
    return query

def delete_composite_index_query_on_id(id):
    query = f"""DROP INDEX composite_range_rel_index__{id} IF EXISTS"""
    return query

def update_relations_query_on_property_id(relation, property_id, parameters):

    query = f"""MATCH ()-[r:{relation} """
    query += """{property_id: """
    query += str(property_id)
    query += """}]->() """

    query += """SET r = {"""

    for item in parameters:
        query += f"""{item}: "{parameters[item]}","""

    query = query[:-1]
    query += """} RETURN COUNT(r)"""
    return query

def get_edge_types(session):
    query = "call db.relationshipTypes()"
    result  =session.run(query)
    return [record["relationshipType"] for record in result]


In [149]:
# Erstellt die Metadaten zu allen Edges, die existieren

conn = Neo4jConnection("bolt://localhost:7687","neo4j","academic")

with conn.driver.session() as session:
    existing_edges = get_edge_types(session)
    for i, item in property.iterrows():
        if item["en_label_text_clean"] in existing_edges:

            session.run(create_composite_index_query_on_id(item["en_label_text_clean"],"property_id"))
        
            res = session.run(update_relations_query_on_property_id(
                relation=item["en_label_text_clean"],
                property_id=item["property_id"],
                parameters={"label": item["en_label"],
                        "description": str(item["en_description"]).replace('"',"'"),
                        }))
        
            session.run(delete_composite_index_query_on_id("property_id"))

Verbindung zur Neo4j-Datenbank erfolgreich hergestellt.
